# 🎤 Codename RVC Fork 4 - Modo GUI (Google Colab)

##### Notebook made by shiroug :shiba:
##### Pretrain custom made by Lyery

> 📁 **Esta versão tem suporte ao Google Drive** para salvar seus modelos e datasets!

In [ ]:
# @title 1️⃣ Montar Google Drive e Clonar Repositório
# @markdown Monta o Drive, clona o repositório e cria symlinks para persistência

MOUNT_DRIVE = True  # @param {type:"boolean"}
DRIVE_FOLDER = "codename-rvc"  # @param {type:"string"}

import os
import shutil
from pathlib import Path

%cd /content

DRIVE_PATH = None

# === MONTAR GOOGLE DRIVE ===
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    DRIVE_PATH = Path(f"/content/drive/MyDrive/{DRIVE_FOLDER}")
    DRIVE_PATH.mkdir(parents=True, exist_ok=True)

    # Criar subpastas no Drive
    (DRIVE_PATH / "datasets").mkdir(exist_ok=True)
    (DRIVE_PATH / "logs").mkdir(exist_ok=True)  # Modelos treinados
    (DRIVE_PATH / "pretrains").mkdir(exist_ok=True)
    (DRIVE_PATH / "outputs").mkdir(exist_ok=True)

    print(f"✅ Google Drive montado em: {DRIVE_PATH}")
else:
    print("⚠️ Drive não montado. Arquivos serão perdidos ao desconectar.")

# === CLONAR REPOSITÓRIO ===
print("\n📥 Clonando repositório...")
!git clone https://github.com/ShiromiyaG/codename-rvc-fork-4.git
%cd codename-rvc-fork-4

# === CRIAR SYMLINKS PARA O DRIVE ===
if MOUNT_DRIVE and DRIVE_PATH:
    print("\n🔗 Criando symlinks para o Drive...")

    # Symlink logs/ -> Drive/logs/ (modelos treinados)
    logs_local = Path("logs")
    logs_drive = DRIVE_PATH / "logs"
    if logs_local.exists() or logs_local.is_symlink():
        if logs_local.is_symlink():
            logs_local.unlink()
        else:
            shutil.rmtree(logs_local)
    os.symlink(logs_drive, logs_local)
    print(f"   ✅ logs/ -> {logs_drive}")

    # Symlink assets/datasets/ -> Drive/datasets/
    datasets_local = Path("assets/datasets")
    datasets_drive = DRIVE_PATH / "datasets"
    datasets_local.parent.mkdir(parents=True, exist_ok=True)
    if datasets_local.exists() or datasets_local.is_symlink():
        if datasets_local.is_symlink():
            datasets_local.unlink()
        else:
            shutil.rmtree(datasets_local)
    os.symlink(datasets_drive, datasets_local)
    print(f"   ✅ assets/datasets/ -> {datasets_drive}")

    # Symlink rvc/models/pretraineds/custom/ -> Drive/pretrains/
    pretrains_local = Path("rvc/models/pretraineds/custom")
    pretrains_drive = DRIVE_PATH / "pretrains"
    pretrains_local.parent.mkdir(parents=True, exist_ok=True)
    if pretrains_local.exists() or pretrains_local.is_symlink():
        if pretrains_local.is_symlink():
            pretrains_local.unlink()
        else:
            shutil.rmtree(pretrains_local)
    os.symlink(pretrains_drive, pretrains_local)
    print(f"   ✅ rvc/models/pretraineds/custom/ -> {pretrains_drive}")

    print("\n📁 Estrutura no Drive:")
    print(f"   {DRIVE_PATH}/")
    print(f"   ├── datasets/   <- Coloque seus áudios aqui")
    print(f"   ├── logs/       <- Modelos treinados (automático)")
    print(f"   ├── pretrains/  <- Pretrains G/D custom")
    print(f"   └── outputs/    <- Áudios convertidos")

# === INSTALAR DEPENDÊNCIAS ===
print("\n🔄 Configurando Python 3.10...")
!apt-get update -qq
!apt-get install -y -qq python3.10 python3.10-dev python3.10-distutils libpython3.10-dev curl aria2
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!update-alternatives --set python3 /usr/bin/python3.10

print("\n📦 Instalando dependências...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
os.environ["PATH"] += ":/root/.cargo/bin"
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3
!uv pip install --system -r requirements.txt
!uv pip install --system pesq ring-attention-pytorch

print("\n✅ Configuração concluída!")
!python3 --version

In [ ]:
# @title 2️⃣ Download Custom Pretrains (G & D)
# @markdown Cole as URLs dos seus arquivos G e D abaixo:
# @markdown ---
# @markdown ### 🎯 Generator (G)
G_url = "https://huggingface.co/lyery/mode4/resolve/main/G_Spin.pth" # @param {type:"string"}
G_filename = "G_Spin.pth" # @param {type:"string"}

# @markdown ---
# @markdown ### 🎯 Discriminator (D)
D_url = "https://huggingface.co/lyery/mode4/resolve/main/D_Spin.pth" # @param {type:"string"}
D_filename = "D_Spin.pth" # @param {type:"string"}

folder = "custom"

import os
import subprocess
import re
from pathlib import Path

def get_gdrive_direct_link(url):
    """Convert Google Drive share link to direct download link."""
    patterns = [
        r'/file/d/([a-zA-Z0-9_-]+)',
        r'id=([a-zA-Z0-9_-]+)',
        r'/open\?id=([a-zA-Z0-9_-]+)',
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            file_id = match.group(1)
            return f"https://drive.google.com/uc?export=download&id={file_id}&confirm=t"
    return url

def get_hf_direct_link(url):
    """Ensure Hugging Face URL is a direct download link."""
    if '/blob/' in url:
        url = url.replace('/blob/', '/resolve/')
    return url

def download_file(url, output_path, filename):
    """Download a file using aria2c."""
    if not url or not url.strip():
        return None

    url = url.strip()

    # Detect and convert URL type
    if 'drive.google.com' in url:
        url = get_gdrive_direct_link(url)
        print(f"   📎 Google Drive detectado")
    elif 'huggingface.co' in url:
        url = get_hf_direct_link(url)
        print(f"   📎 Hugging Face detectado")

    output_path.mkdir(parents=True, exist_ok=True)
    full_path = output_path / filename

    if full_path.exists():
        size_mb = full_path.stat().st_size / (1024 * 1024)
        print(f"   ⏭️ Já existe ({size_mb:.1f} MB)")
        return True

    cmd = [
        'aria2c', '-x', '16', '-s', '16', '-j', '1', '-c',
        '--file-allocation=none',
        '-d', str(output_path), '-o', filename, url
    ]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0 and full_path.exists():
            size_mb = full_path.stat().st_size / (1024 * 1024)
            print(f"   ✅ Baixado ({size_mb:.1f} MB)")
            return True
        else:
            # Fallback wget
            print(f"   🔄 Tentando wget...")
            subprocess.run(['wget', '-q', '-O', str(full_path), url])
            if full_path.exists():
                size_mb = full_path.stat().st_size / (1024 * 1024)
                print(f"   ✅ Baixado com wget ({size_mb:.1f} MB)")
                return True
            print(f"   ❌ Falha no download")
            return False
    except Exception as e:
        print(f"   ❌ Erro: {e}")
        return False

# Folder mapping
folder_map = {
    'pretrains': Path('rvc/models/pretrains'),
    'pretrains_v2': Path('rvc/models/pretrains_v2'),
    'custom': Path('rvc/models/pretraineds/custom'),
}

print("📥 Download de Pretrains Customizados")
print("="*60)

target_folder = folder_map.get(folder, folder_map['pretrains'])
print(f"📁 Pasta destino: {target_folder}")
print()

# Download G
if G_url:
    print(f"🎯 Generator: {G_filename}")
    download_file(G_url, target_folder, G_filename)
else:
    print("⚠️ URL do Generator (G) não fornecida")

print()

# Download D
if D_url:
    print(f"🎯 Discriminator: {D_filename}")
    download_file(D_url, target_folder, D_filename)
else:
    print("⚠️ URL do Discriminator (D) não fornecida")

print()
print("="*60)

# List downloaded files
print("📁 Arquivos na pasta:")
if target_folder.exists():
    for f in target_folder.glob("*.pth"):
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"   • {f.name} ({size_mb:.1f} MB)")
else:
    print("   (pasta vazia)")

In [ ]:
# @title 3️⃣ Iniciar a Interface Gráfica (GUI)
!python app.py --share